---
### gensim simple_preprocess

In [16]:
import pandas as pd

# 데이터 로딩
data = pd.read_csv("./total_naver.csv")

# 필요한 컬럼만 선택
docs = data["content"].tolist()
from gensim.utils import simple_preprocess

# 텍스트 토큰화
tokenized_docs = [simple_preprocess(doc) for doc in docs]
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# TaggedDocument 객체 생성
tagged_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized_docs)]

# Doc2Vec 모델 생성
model = Doc2Vec(vector_size=300, min_count=2, epochs=40, workers=8)

# 모델 학습
model.build_vocab(tagged_docs)
model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)
model.save('naver.doc2vec')

1000개 학습 -> 13초
총 397639개 학습 필요

In [ ]:
similar_docs = model.dv.most_similar(39000)

In [34]:
similar_docs

[(63078, 0.5626869797706604),
 (60142, 0.5477779507637024),
 (63037, 0.5389310121536255),
 (32618, 0.5368140935897827),
 (395518, 0.5305687189102173),
 (60140, 0.5299805402755737),
 (63024, 0.5279240012168884),
 (25223, 0.5224257707595825),
 (38976, 0.5219411253929138),
 (36038, 0.5195015668869019)]

In [169]:
def show_results(similar_docs):
    result = pd.DataFrame()
    for idx, val in similar_docs:
        temp = data.iloc[[idx]]
        temp.loc[:, 'cos_simil'] = val
        result = pd.concat([result, temp], axis=0)
    return result.loc[:, ['title','date_upload','content', 'cos_simil']]

In [170]:
data.iloc[[39000]] 

,cat1_name,cat2_name,platform_name,title,press,writer,date_upload,content,sticker,url
cat2_id,,,,,,,,,,
20202,경제,증권,네이버,"""악성 문서, 12초면 찾는다""…코스닥 출사표",한국경제TV,박승완 기자,2023-08-07 12:39:00,2026년까지 약 20조 원 규모로 성장이 예상되는 글로벌 지능형 보안위협 대응 ...,"""{'wow': 0, 'useful': 1, 'touched': 0, 'analyt...",https://n.news.naver.com/mnews/article/215/000...


In [175]:
show_results(similar_docs)

,title,date_upload,content,cos_simil
cat2_id,,,,
20202,"시큐레터, 증권신고서 제출…8월 코스닥 상장",2023-06-30 16:02:00,[서울=뉴시스] 김경택 기자 = 사이버 보안 전문 기업 시큐레터는 30일 증권신고...,0.562687
20202,코스피 하락에 2570선,2023-07-05 16:03:00,[서울=뉴시스] 최동준 기자 = 코스피가 전 거래일보다 14.31포인트(0.55%...,0.547778
20202,"시큐레터, 증권신고서 제출…코스닥 상장 준비 '본격화'",2023-06-30 16:26:00,사이버 보안 전문 기업 시큐레터는 증권신고서를 제출하고 코스닥 시장 상장을 위한 ...,0.538931
20202,"시큐레터, 일반청약 흥행…경쟁률 1698대 1 기록",2023-08-16 17:21:00,[이코노미스트 마켓in 송재민 기자] 사이버 보안 전문기업 시큐레터가 지난 14일...,0.536814
20206,[유럽개장]장 초반 강보합세…英 0.11%↑,2023-07-10 17:16:00,10일 유럽 주요 증시가 장 초반 강보합세를 나타내고 있다. 이날 오후 5시13분(...,0.530569
20202,코스피 하락 마감,2023-07-05 16:02:00,[서울=뉴시스] 최동준 기자 = 코스피가 전 거래일보다 14.31포인트(0.55%...,0.529981
20202,시큐레터 8월 상장 첫발…최대 122억원 공모,2023-06-30 16:50:00,사이버 보안 전문 기업 시큐레터가 증권신고서를 제출하고 코스닥 시장 상장을 위한 ...,0.527924
20202,"""공공시장 공략한다""... 비트나인, ‘아젠스SQL’ 조달청 디지털서비스몰 등록",2023-08-29 10:37:00,[파이낸셜뉴스] 비트나인이 오픈소스 포스트그레SQL 기반 엔터프라이즈급 데이터베...,0.522426
20202,"시큐레터 ""악성 코드 선제 방어…정보보안 전문기업될 것""",2023-08-07 13:19:00,사이버 보안 전문기업 시큐레터가 공공시장과 민간시장을 아우르는 정보보안 전문기업으...,0.521941
